### 使用Functional API打造神經網路，加一些分岔的部分在模型結構中

In [2]:
%env KERAS_BACKEND=tensorflow
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact_manual
from ipywidgets import interact, IntSlider, Button

env: KERAS_BACKEND=tensorflow


In [3]:
# Keras functions
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD,Adam

# Keras dataset
from keras.datasets import mnist

# Keras utils
from keras.utils import np_utils


from keras.models import Model
from keras.layers import Input
from keras.layers import concatenate, add

d:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


讀入MNIST的資料，然後把資料變成我們要的樣子

In [4]:
(x_train0, y_train0), (x_test0, y_test0) = mnist.load_data()

In [5]:
x_train = x_train0.reshape(60000, 28*28)
x_test = x_test0.reshape(10000, 28*28)

In [6]:
y_train= np_utils.to_categorical(y_train0, 10)
y_test= np_utils.to_categorical(y_test0, 10)

用Functional API建立神經網路

In [7]:
x = Input(shape=(784,))
f_1_1 = Dense(200, activation='sigmoid')
f_1_2 = Dense(200, activation='relu')
f_1_3 = Dense(200, activation='sigmoid')

f_2_1 = Dense(100, activation='sigmoid')
f_2_2 = Dense(100, activation='sigmoid')
f_2_3 = Dense(100, activation='sigmoid')
f_2_4 = Dense(100, activation='sigmoid')


f_3_1 = Dense(50, activation='sigmoid')
f_3_2 = Dense(50, activation='sigmoid')
f_3_3 = Dense(50, activation='sigmoid')

f_4 = Dense(10, activation='softmax')

In [8]:
h_1_1 = f_1_1(x)
h_1_2 = f_1_2(x)
h_1_3 = f_1_3(x)
h_1_4 = concatenate([h_1_1,h_1_2])


h_2_1 = f_2_1(h_1_4)
h_2_2 = f_2_2(h_1_4)
h_2_3 = f_2_3(h_1_3)
h_2_4 = f_2_4(h_1_3)
h_2_5 = concatenate([h_2_1,h_2_2,h_2_3])
h_2_6 = concatenate([h_2_1,h_2_4])

h_3_1 = f_3_1(h_2_5)
h_3_2 = f_3_2(h_2_5)
h_3_3 = f_3_3(h_2_6)
h_3 = concatenate([h_3_1,h_3_2,h_3_3])

y = f_4(h_3)

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
model=Model(x,y)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 784)          0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 200)          157000      input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 200)          157000      input_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 400)          0           dense_1[0][0]                    
                                                                 dense_2[0][0]                    
__________

In [10]:
model.compile(loss='mse', optimizer=SGD(lr=0.087), metrics=['accuracy'])

訓練並看一下成果

In [13]:
model.fit(x_train,y_train,batch_size=100, epochs=20)

Epoch 1/20
60000/60000 [==============================] - 7s 123us/step - loss: 0.0360 - acc: 0.8188
Epoch 2/20
60000/60000 [==============================] - 8s 131us/step - loss: 0.0288 - acc: 0.8689 1s - 
Epoch 3/20
60000/60000 [==============================] - 8s 133us/step - loss: 0.0235 - acc: 0.8916
Epoch 4/20
60000/60000 [==============================] - 8s 136us/step - loss: 0.0199 - acc: 0.9009
Epoch 5/20
60000/60000 [==============================] - 8s 127us/step - loss: 0.0174 - acc: 0.9085
Epoch 6/20
60000/60000 [==============================] - 8s 134us/step - loss: 0.0156 - acc: 0.9149
Epoch 7/20
60000/60000 [==============================] - 9s 149us/step - loss: 0.0143 - acc: 0.9198 0s - loss: 0.0143 - acc: 0.91
Epoch 8/20
60000/60000 [==============================] - 8s 142us/step - loss: 0.0133 - acc: 0.9247
Epoch 9/20
60000/60000 [==============================] - 8s 138us/step - loss: 0.0124 - acc: 0.9286
Epoch 10/20
60000/60000 [==============================

In [14]:
score = model.evaluate(x_test, y_test, batch_size=10000)
print("Loss: %f" %score[0])
print("準確率: %f" %(score[1]*100))

10000/10000 [==============================] - 0s 32us/step
Loss: 0.008623
準確率: 94.569999
